In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image

import tensorflow as tf
from mobilenet_v2 import MobileNetv2

from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, Reshape, Activation
from keras.models import Model

In [ ]:
DATA_PATH = '../../data/input/PublicTest'
WEIGHT_PATH = 'model/weights_face_grapy96_30eps.h5'
DATA_SIZE = 96

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)
flow = datagen.flow_from_directory(
        DATA_PATH,
        target_size=(DATA_SIZE, DATA_SIZE),
        batch_size=1,
        color_mode="grayscale",
        class_mode='categorical')
                             
print (flow.class_indices)

In [ ]:
model = MobileNetv2((DATA_SIZE,DATA_SIZE,1),len(flow.class_indices))

In [ ]:
model.load_weights(WEIGHT_PATH)
opt = Adam()
earlystop = EarlyStopping(monitor='val_acc', patience=30, verbose=0, mode='auto')
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

def label_from_raw(labels, raw_values):
    idx = np.argmax(raw_values)
    return get_keys_from_value(labels, idx)

In [ ]:
model._make_predict_function()

image_batch, label_batch = next(flow)
pred_raw = model.predict(image_batch)
idx = np.argmax(pred_raw)
pred_label = label_from_raw(flow.class_indices, pred_raw)
true_label = label_from_raw(flow.class_indices, label_batch[0])

fix, ax = plt.subplots()
ax.imshow(image_batch[0].reshape([DATA_SIZE, DATA_SIZE]), cmap="gray")
plt.show()
print("truth", true_label)
print("prediction", pred_label)

In [ ]:
model.save('model.h5')